In [2]:
import requests
from bs4 import BeautifulSoup
from bs4 import NavigableString, Tag
import pandas as pd
from textblob import TextBlob
import wordninja
from datetime import datetime

In [82]:
url = "https://de.wikipedia.org/wiki/Liste_der_St%C3%A4dte_in_Deutschland"

page = requests.get(url)    
data = page.text
soup = BeautifulSoup(data)

cont = soup.find_all('dd')

cities_list = []

for c in cont:
    cities_list.append(c.text)

In [83]:
cities_list = [x.rsplit(' ', 1) for x in cities_list]

In [89]:
list_of_eastern_terr = ['(MV)', '(BB)', '(BE)', '(SN)', '(ST)', '(TH)']

df = pd.DataFrame({'City' : [x[0] for x in cities_list],
                  'Land': ['Eastern' if x[1] in list_of_eastern_terr else 'Western' for x in cities_list] 
                  })

In [94]:
df = df.append({'City' : 'Munich',
               'Land' : 'Western'
               }, 
              ignore_index=True)

df.to_csv('list_of_cities_ext')

In [8]:
cities_list

[]

In [93]:
url = "https://simple.wikipedia.org/wiki/List_of_countries_by_continents"

page = requests.get(url)    
data = page.text
soup = BeautifulSoup(data)

ord_cont = soup.find_all('ol')
unord_cont = soup.find_all('ul')

list_of_count = []
list_of_continents = []

for c in ord_cont:
    li_sm = []
    list_of_count.append(li_sm)
    for li in c.findAll('li'):
        try:
            li_sm.append(li.find('a').text)
        except AttributeError:
            pass
        
for c in unord_cont:
    for li in c.findAll('li'):
        for a in li.findAll('a'):
            list_of_continents.append(a.text)

In [94]:
list_of_continents_upd = list_of_continents[:1] + list_of_continents[2:7] 
list_of_continents_upd = [(i).split(' ', 1)[1] for i in list_of_continents_upd]

#for_deletion = ['[1]', 'বাংলাদেশ', 'ঢাকা', 'বাংলাদেশ']
#for ele in list_of_count[1]:
#    if ele in for_deletion: 
#        list_of_count[1].remove(ele)
        
#list_of_count[1].remove('বাংলাদেশ')

In [95]:
list_of_count = list_of_count[:6]

In [96]:
list_of_continents_upd

['Africa',
 'Asia',
 'Europe',
 'North America',
 'South America',
 'Australia/Oceania']

In [97]:
dict_of_countries_cont = {k:v for k, v in zip(list_of_continents_upd, list_of_count)}

In [98]:
dict_of_countries_cont

{'Africa': ['Algeria',
  'Angola',
  'Benin',
  'Botswana',
  'Burkina Faso',
  'Burundi',
  'Cameroon',
  'Cape Verde',
  'Central African Republic',
  'Chad',
  'Comoros',
  'Republic of the Congo',
  'Democratic Republic of the Congo',
  "Côte d'Ivoire",
  'Djibouti',
  'Egypt',
  'Equatorial Guinea',
  'Eritrea',
  'Ethiopia',
  'Gabon',
  'The Gambia',
  'Ghana',
  'Guinea',
  'Guinea-Bissau',
  'Kenya',
  'Lesotho',
  'Liberia',
  'Libya',
  'Madagascar',
  'Malawi',
  'Mali',
  'Mauritania',
  'Mauritius',
  'Morocco',
  'Mozambique',
  'Namibia',
  'Niger',
  'Nigeria',
  'Rwanda',
  'São Tomé and Príncipe',
  'Senegal',
  'Seychelles',
  'Sierra Leone',
  'Somalia',
  'South Africa',
  'South Sudan',
  'Sudan',
  'Swaziland',
  'Tanzania',
  'Togo',
  'Tunisia',
  'Uganda',
  'Western Sahara',
  'Zambia',
  'Zimbabwe'],
 'Asia': ['Afghanistan',
  'Armenia',
  'Azerbaijan',
  'Bahrain',
  'Bangladesh',
  'Bhutan',
  'Brunei',
  'Cambodia',
  'China',
  'East Timor',
  'Georgia'

In [85]:
df = pd.read_csv('geotag_final')

In [87]:
df = df[df['countries'].notna()]

In [88]:
def us_edit(row):
    if row['countries'] == 'United States':
        return'United States of America'
    return row['countries']

In [89]:
df['countries'] = df.apply(lambda row: us_edit(row), axis=1)

C:\Users\wedan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [99]:
def label_cont(row):
        is_found = False
        for k, v in dict_of_countries_cont.items():
            if row['countries'] in v:
                is_found = True
                return k
                break
        if not is_found:
            return None

In [100]:
df['cont'] = df.apply(lambda row: label_cont(row), axis=1)

C:\Users\wedan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [101]:
df

,Unnamed: 0,created_at,full_text,lang,place_type,cities,countries,cont
0,0,2020-08-10 04:38:13+00:00,Awesome BRAVE SkillAthletic class with Joe at ...,en,city,North Lambton,Australia,Australia/Oceania
1,1,2020-08-10 06:06:50+00:00,"So I love working from home, I don't need to w...",en,city,Clerkenwell,United Kingdom,Europe
2,2,2020-08-10 04:44:47+00:00,Matching our swimsuits may have been a step to...,en,city,Dallas,United States of America,North America
3,3,2020-08-10 06:18:56+00:00,1st staying overnight in a tent! Social distan...,en,admin,Livingston,United States of America,North America
4,4,2020-08-10 05:03:14+00:00,Failed #FieldWork. Resident did not let us in ...,en,city,Malanday,Philippines,Asia
...,...,...,...,...,...,...,...,...
239113,239113,2020-09-10 04:19:19+00:00,Don’t judge me but during this #RonaDivide &am...,en,city,Bowie,United States of America,North America
239114,239114,2020-09-10 04:00:04+00:00,#RT #Trump Claims He Downplayed COVID-19 To Av...,en,city,Warsaw,Poland,Europe
239115,239115,2020-09-10 04:12:32+00:00,Number of coronavirus cases in #Germany rises ...,en,city,London,United Kingdom,Europe
239116,239116,2020-09-10 04:18:10+00:00,"What lockdown? 🧔keep smiling guys, we can do t...",en,city,Melbourne,Australia,Australia/Oceania


In [102]:
df['countries'].isnull().values.any()

False

In [103]:
df.to_csv('data+continents')

In [106]:
df1 = pd.read_csv('data+continents')
df1.drop(['Unnamed: 0','Unnamed: 0.1'], axis = 1, inplace = True)

In [107]:
df

,created_at,full_text,lang,place_type,cities,countries,cont
0,2020-08-10 04:38:13+00:00,Awesome BRAVE SkillAthletic class with Joe at ...,en,city,North Lambton,Australia,Australia/Oceania
1,2020-08-10 06:06:50+00:00,"So I love working from home, I don't need to w...",en,city,Clerkenwell,United Kingdom,Europe
2,2020-08-10 04:44:47+00:00,Matching our swimsuits may have been a step to...,en,city,Dallas,United States of America,North America
3,2020-08-10 06:18:56+00:00,1st staying overnight in a tent! Social distan...,en,admin,Livingston,United States of America,North America
4,2020-08-10 05:03:14+00:00,Failed #FieldWork. Resident did not let us in ...,en,city,Malanday,Philippines,Asia
...,...,...,...,...,...,...,...
238995,2020-09-10 04:19:19+00:00,Don’t judge me but during this #RonaDivide &am...,en,city,Bowie,United States of America,North America
238996,2020-09-10 04:00:04+00:00,#RT #Trump Claims He Downplayed COVID-19 To Av...,en,city,Warsaw,Poland,Europe
238997,2020-09-10 04:12:32+00:00,Number of coronavirus cases in #Germany rises ...,en,city,London,United Kingdom,Europe
238998,2020-09-10 04:18:10+00:00,"What lockdown? 🧔keep smiling guys, we can do t...",en,city,Melbourne,Australia,Australia/Oceania


In [108]:
url = "https://en.wikipedia.org/wiki/List_of_countries_by_largest_and_second_largest_cities"

page = requests.get(url)    
data = page.text
soup = BeautifulSoup(data)

cont = soup.find_all('tr')

countries = []
big_citi = []
for tr in cont[4:]:
    try:
        tds = tr.find_all('td')
        countries.append(tds[0].text)
        big_citi.append([tds[1].text, tds[2].text])
    except IndexError:
        countries.remove(tds[0].text)
        pass

In [109]:
dic = {'\xa0':'',
      '\n':''}

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

countries = [replace_all(i, dic) for i in countries]
big_cities = [[i.replace('\n',' ') for i in inner_list] for inner_list in big_citi]

In [110]:
dic_count_big_cities = dict(zip(countries,big_cities))

In [111]:
li_cities_type = []

for i, j in zip(df['countries'], df['cities']):
    isFound = False
    for key,value in dic_count_big_cities.items():
        if i == key and j in value:
            isFound = True
            li_cities_type.append('big')
            break
    if not isFound:
        li_cities_type.append('small')

In [119]:
df['city_size'] = li_cities_type
df.to_csv('data+cont+cities', index = False)

In [8]:
df = pd.read_csv('data+cont+cities')

In [9]:
df

,created_at,full_text,lang,place_type,cities,countries,cont,city_size
0,2020-08-10 04:38:13+00:00,Awesome BRAVE SkillAthletic class with Joe at ...,en,city,North Lambton,Australia,Australia/Oceania,small
1,2020-08-10 06:06:50+00:00,"So I love working from home, I don't need to w...",en,city,Clerkenwell,United Kingdom,Europe,small
2,2020-08-10 04:44:47+00:00,Matching our swimsuits may have been a step to...,en,city,Dallas,United States of America,North America,small
3,2020-08-10 06:18:56+00:00,1st staying overnight in a tent! Social distan...,en,admin,Livingston,United States of America,North America,small
4,2020-08-10 05:03:14+00:00,Failed #FieldWork. Resident did not let us in ...,en,city,Malanday,Philippines,Asia,small
...,...,...,...,...,...,...,...,...
238995,2020-09-10 04:19:19+00:00,Don’t judge me but during this #RonaDivide &am...,en,city,Bowie,United States of America,North America,small
238996,2020-09-10 04:00:04+00:00,#RT #Trump Claims He Downplayed COVID-19 To Av...,en,city,Warsaw,Poland,Europe,big
238997,2020-09-10 04:12:32+00:00,Number of coronavirus cases in #Germany rises ...,en,city,London,United Kingdom,Europe,big
238998,2020-09-10 04:18:10+00:00,"What lockdown? 🧔keep smiling guys, we can do t...",en,city,Melbourne,Australia,Australia/Oceania,big


In [12]:
text = df['full_text'].tolist()

In [13]:
def label_sent(li):
    return [TextBlob(' '.join(wordninja.split(i))).sentiment.polarity for i in li]

In [16]:
df['sentiment'] = label_sent(text)

In [18]:
df.to_csv('covid_data_final+cont')

In [20]:
df1 = pd.read_csv('covid_data_final+cont')
df1

,Unnamed: 0,created_at,full_text,lang,place_type,cities,countries,cont,city_size,sentiment
0,0,2020-08-10 04:38:13+00:00,Awesome BRAVE SkillAthletic class with Joe at ...,en,city,North Lambton,Australia,Australia/Oceania,small,0.800000
1,1,2020-08-10 06:06:50+00:00,"So I love working from home, I don't need to w...",en,city,Clerkenwell,United Kingdom,Europe,small,0.500000
2,2,2020-08-10 04:44:47+00:00,Matching our swimsuits may have been a step to...,en,city,Dallas,United States of America,North America,small,0.440000
3,3,2020-08-10 06:18:56+00:00,1st staying overnight in a tent! Social distan...,en,admin,Livingston,United States of America,North America,small,0.033333
4,4,2020-08-10 05:03:14+00:00,Failed #FieldWork. Resident did not let us in ...,en,city,Malanday,Philippines,Asia,small,-0.312500
...,...,...,...,...,...,...,...,...,...,...
238995,238995,2020-09-10 04:19:19+00:00,Don’t judge me but during this #RonaDivide &am...,en,city,Bowie,United States of America,North America,small,0.320833
238996,238996,2020-09-10 04:00:04+00:00,#RT #Trump Claims He Downplayed COVID-19 To Av...,en,city,Warsaw,Poland,Europe,big,0.000000
238997,238997,2020-09-10 04:12:32+00:00,Number of coronavirus cases in #Germany rises ...,en,city,London,United Kingdom,Europe,big,0.000000
238998,238998,2020-09-10 04:18:10+00:00,"What lockdown? 🧔keep smiling guys, we can do t...",en,city,Melbourne,Australia,Australia/Oceania,big,0.227273


In [21]:
df1['created_at'] = pd.to_datetime(df1['created_at'])

In [22]:
df1.sort_values(by='created_at', inplace = True)

In [24]:
df1.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [25]:
df1

,created_at,full_text,lang,place_type,cities,countries,cont,city_size,sentiment
143381,2020-03-19 19:53:17+00:00,#statewaterheaters #getitin #corona #keepingpe...,en,city,Pickerington,United States of America,North America,small,0.318182
143413,2020-03-19 19:53:46+00:00,"""ain't no humans outside! (corona!)"" 😂😂😂🤣 @ Cl...",en,city,Clark-Fulton,United States of America,North America,small,0.000000
143383,2020-03-19 19:53:54+00:00,Salam Friends\nLooking at the grave financial ...,en,city,Karachi,Pakistan,Asia,big,-0.006250
143407,2020-03-19 19:54:27+00:00,Thanks to COVID19 we are under unprecedented l...,en,city,Meyerton,South Africa,Africa,small,0.275000
143429,2020-03-19 19:55:33+00:00,#tbt to the current #anime that I'm rewatching...,en,admin,Ouésso,Congo,NaN,small,0.000000
...,...,...,...,...,...,...,...,...,...
121054,2020-11-11 03:49:10+00:00,Bodes well for social distancing...\n\nThe Mas...,en,city,Kunnamangalam,India,Asia,small,0.033333
120997,2020-11-11 03:56:13+00:00,@mtgreenee You are fucking stupid. We are in a...,en,city,Salt Lake City,United States of America,North America,small,-0.175000
121065,2020-11-11 03:57:20+00:00,"Your Bonjour Minnesota on KFAI Team, bringing ...",en,city,Columbus,United States of America,North America,small,0.234091
121042,2020-11-11 04:05:18+00:00,Grateful to see another year and for the healt...,en,city,Subic,Philippines,Asia,small,0.750000


In [31]:
cont = set(df['city_size'])

for i in cont:
    print(i, len(df[df['city_size'] == i]))

small 214891
big 24109


In [26]:
df.to_csv('covid_data_final+cont', index = False)